In [ ]:
"""
solar elongation angle, epsilon ***
    date, time
ecliptic latitude, beta
    ra, dec, time, date
fractional variability, Fs
    V - variance
    sigma_err - rms error
    S - flux density
        Jy/beam -> beam calculaiton
variability index, Vs
    S - flux density
    sigma - S error
modulation index, M
    sigma_S - stddev of S
    S_bar - mean S
epoch
solar activity
    Y/N, 0/1
rms noise
    
phase error ratios?
"""

In [ ]:
"""
list of var's:
    x3*1.5mil:
        peak flux
        err peak flux
        total flux
        err total flux
        island total flux
        err island total flux
        SE
        EL
        isl rms

    x3*3:
        V^2
        RMSE
        S_bar
        F_s
        delta_F_s
        V_s
        sigma_s
        M
        rms_noise
        solar activity

correlation list:
    flux (all 3 + errors)
    SE
    EL
    4 statistical vars
    ra/dec
    solar activity
    rms noise
    epoch
"""

In [ ]:
"""
script:
    append:
        SE 1/2/3 [get_sun(t)]
        EL 1/2/3 [GeocentricMeanEcliptic()]
        solar activity (t) 1/2/3
    get:
        F_s (peak, tot, isl)
        deltaF_s (peak, tot, isl)
        V_s (peak, tot, isl)
        M (peak, tot, isl)
    correl. matrix:
        3 fluxes
        3 errs
        SE
        EL
        solar activity
        rms noise/phase errors*
"""

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord, GeocentricMeanEcliptic

# Example equatorial coordinates (Right Ascension and Declination)
ra = 10.625 * u.degree
dec = 41.2 * u.degree

# Create a SkyCoord object with equatorial coordinates
coord_eq = SkyCoord(ra=ra, dec=dec, frame='icrs')

# Transform to geocentric mean ecliptic coordinates
coord_ecl = coord_eq.transform_to(GeocentricMeanEcliptic())

coord_ecl.lat.degree

In [ ]:
from astropy.time import Time
from astropy.coordinates import get_sun
from astropy.coordinates import EarthLocation

time = Time('2025-06-03 12:00:00')

get_sun(time)

In [ ]:
import matplotlib.pyplot as plt
import sunpy.timeseries as ts
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.time import TimeRange

# Define a time range
tr = TimeRange("1900-01-01", "2025-01-01")

# Search for the sunspot number data
results = Fido.search(a.Time(tr), a.Instrument("SIS"), a.Source("SWPC"))

# Download the data
files = Fido.fetch(results)

# Load the data into a TimeSeries object
ts_ = ts.TimeSeries(files)

# Plot the sunspot number
ts_.plot()
plt.show()


In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord, get_sun, GeocentricMeanEcliptic
from astropy.time import Time
import astropy.units as u

csv_path = "combined_data2.csv"
df = pd.read_csv(csv_path)

ra_cols = []
dec_cols = []
date_cols = []

for i in range(1, 4):
    ra_name = f"RA_{i}"
    dec_name = f"DEC_{i}"
    date_name = f"Epoch_{i}_date-obs"
    
    if ra_name not in df.columns:
        raise KeyError(f"Column '{ra_name}' not found in CSV")
    if dec_name not in df.columns:
        raise KeyError(f"Column '{dec_name}' not found in CSV")
    if date_name not in df.columns:
        raise KeyError(f"Column '{date_name}' not found in CSV")
        
    ra_cols.append(ra_name)
    dec_cols.append(dec_name)
    date_cols.append(date_name)
    
for i in range(1, 4):
    df[f"Epoch_{i}_solar_elongation"] = pd.NA
    df[f"Epoch_{i}_ecliptic_latitude"] = pd.NA
    
def compute_angles(ra_deg, dec_deg, timestamp):
    if pd.isna(ra_deg) or pd.isna(dec_deg) or not isinstance(timestamp, str):
        return None, None
        
    if "." in timestamp:
        timestamp = timestamp.split(".", 1)[0]
        
    try:
        t = Time(timestamp)
        target = SkyCoord(ra=ra_deg * u.deg, dec=dec_deg * u.deg, frame="icrs")
        sun = get_sun(t)
        elong = sun.separation(target).deg
        ecl = target.transform_to(GeocentricMeanEcliptic(equinox=t))
        lat = ecl.lat.deg
        return round(elong, 2), round(lat, 2)
        
    except Exception as e:
        print(f"[WARNING] RA={ra_deg}, DEC={dec_deg}, time={timestamp}: {e}")
        return None, None
        
for idx, row in df.iterrows():
    for i in range(3):
        ra_val = row[ra_cols[i]]
        dec_val = row[dec_cols[i]]
        time_str = row[date_cols[i]]
        elong, lat = compute_angles(ra_val, dec_val, time_str)

        df.at[idx, f"Epoch_{i+1}_solar_elongation"] = elong
        df.at[idx, f"Epoch_{i+1}_ecliptic_latitude"] = lat

df.to_csv(csv_path, index=False)
print("updated")